In [1]:
# Load library
import os

from scapy.all import *

In [20]:
# read pcap files
def read_pcap(root_dir, ext=('.pcap', '.pcapng')):
    queue = [root_dir]
    while len(queue) != 0:
        nest_dir = queue.pop()
        with os.scandir(nest_dir) as it:
            for entry in it:
                if not entry.name.startswith('.') and entry.is_file():
                    if entry.name.endswith(ext):
                        label = os.path.basename(os.path.dirname(entry.path)) # dirname is label
                        yield label, entry.path
                elif not entry.name.startswith('.') and entry.is_dir():
                    queue.append(entry.path)

In [28]:
# Get sample pcap path
pcap_path = os.path.abspath(os.path.expanduser('./data'))
for path in read_pcap(pcap_path):
    tpath = path
    break
print(f'Sample pcap path: {tpath[1]}')

Sample pcap path: /home/harny/Github/tff-app/data/facebook/facebook_audio1a.pcap


In [22]:
# pcap to vector
## pkt['Ether'].type == 0x0800 # IPv4 Packet
def pcap2vec(pcap):
    rp = rdpcap(pcap)
    fv = ''
    for key, value in rp.sessions().items():
        protocol = key.split(' ')[0]
        for pkt in value:
            if pkt['Ether'].type != 2048:
                continue
            print(protocol)
            if protocol == 'UDP':
                feature = get_udp_vector(pkt)
            elif protocol == 'TCP':
                feature = get_tcp_vector(pkt)
        assert len(feature) == 56, f'Feature len: {len(feature)}'
        fv = fv + feature
        if len(fv) >= 1568:
            break
    if len(fv) < 2880:
        padding = 2880 - len(fv)
        fv = fv + '0'*padding
    return fv

In [24]:
rp = rdpcap(tpath[1])
for key, value in rp.sessions().items():
    v = value
    break

In [34]:
for pkt in v:
    print(pkt['Ether'].type == 0x0800)

True
True
